# Campus_GPT Fine-tuning with Unsloth (Local GPU)

This notebook fine-tunes Llama 3.1 8B on the generated RAFT dataset using your local GPU.

**Requirements:**
- CUDA-capable GPU (12GB+ VRAM recommended)
- Python 3.10+
- CUDA Toolkit installed

**Setup:**
```bash
# Install dependencies with uv
uv pip install "unsloth[cu124] @ git+https://github.com/unslothai/unsloth.git"
uv pip install transformers==4.56.2 trl==0.22.2 datasets accelerate bitsandbytes
```

## 1. Load Model with 4-bit Quantization

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # Supports RoPE Scaling
dtype = None  # Auto-detect (Float16 for older GPUs, Bfloat16 for Ampere+)
load_in_4bit = True  # 4-bit quantization to fit in 12GB VRAM

print("Loading Llama 3.1 8B with 4-bit quantization...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✅ Model loaded successfully!")

## 2. Define RAFT Prompt Template

This template includes:
- **Context**: Oracle + Distractors
- **Question**: User query
- **Thought Process**: Chain-of-Thought reasoning
- **Answer**: Final response

In [ ]:
# Llama 3.1 format with thinking tags
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are Campus_GPT, an AI assistant for Northern Kentucky University. 
Use the provided context to answer questions. Show your reasoning before the final answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|thought|>
{thought}
<|answer|>
{answer}<|eot_id|>"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    """
    Map RAFT dataset to Llama 3.1 format.
    
    RAFT keys:
    - question: User query
    - oracle: The correct context chunk
    - distractors: List of similar but incorrect chunks
    - thought_process: Chain-of-Thought reasoning
    - answer: Final answer
    """
    instructions = examples["question"]
    oracles = examples["oracle"]
    distractors_list = examples["distractors"]
    thoughts = examples["thought_process"]
    answers = examples["answer"]
    
    texts = []
    for instruction, oracle, distractors, thought, answer in zip(
        instructions, oracles, distractors_list, thoughts, answers
    ):
        # Combine Oracle + Distractors
        if isinstance(distractors, list):
            all_contexts = [oracle] + distractors[:3]  # Oracle + 3 distractors
        else:
            all_contexts = [oracle]
        
        context_str = "\n\n".join([f"Document {i+1}:\n{c}" for i, c in enumerate(all_contexts)])
        
        text = prompt_template.format(
            context=context_str,
            instruction=instruction,
            thought=thought,
            answer=answer
        ) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}


print("✅ Prompt template defined")

## 3. Load RAFT Dataset

In [ ]:
from datasets import load_dataset
import os

# Direct path to raft_dataset.jsonl in 03_fine_tuning
dataset_path = "raft_dataset.jsonl"

if not os.path.exists(dataset_path):
    raise FileNotFoundError(
        f"❌ {dataset_path} not found!\n"
        "Make sure you've run: python generate_raft_focused.py"
    )

print(f"Loading dataset from {dataset_path}...")
dataset = load_dataset("json", data_files=dataset_path, split="train")

print(f"✅ Loaded {len(dataset)} examples")
print(f"\nSample keys: {dataset.column_names}")

# Apply prompt formatting
print("\nApplying prompt template...")
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"✅ Dataset ready for training ({len(dataset)} examples)")

## 4. Add LoRA Adapters

LoRA allows efficient fine-tuning by updating only 1-10% of parameters.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank (higher = more parameters)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,  # 0 is optimized
    bias="none",
    use_gradient_checkpointing="unsloth",  # 30% less VRAM
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA adapters added")

## 5. Configure Training

These settings are optimized for 12GB VRAM.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,       # Batch size per GPU
        gradient_accumulation_steps=4,       # Effective batch = 2*4 = 8
        warmup_steps=5,
        max_steps=300,                       # Increase for longer training
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),  # Use fp16 if bf16 not available
        bf16=torch.cuda.is_bf16_supported(),       # Use bf16 on Ampere+ GPUs
        logging_steps=10,
        optim="adamw_8bit",                  # Saves ~2GB VRAM
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=50,                       # Save checkpoint every 50 steps
        save_total_limit=3,                  # Keep only 3 checkpoints
    ),
)

print("✅ Trainer configured")

## 6. Check GPU Memory

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU: {gpu_stats.name}")
print(f"Max memory: {max_memory} GB")
print(f"Reserved: {start_gpu_memory} GB")
print(f"Available: {max_memory - start_gpu_memory} GB")

## 7. Train!

This will take ~30-60 minutes depending on your GPU and dataset size.

In [ ]:
print("🚀 Starting training...\n")

trainer_stats = trainer.train()

print("\n✅ Training complete!")
print(f"Final loss: {trainer_stats.training_loss:.4f}")

## 8. Test Inference

In [ ]:
# Enable fast inference mode
FastLanguageModel.for_inference(model)

# Test question
test_context = """Document 1:
NKU tuition for undergraduate students is $450 per credit hour for in-state students and $750 for out-of-state.

Document 2:
Financial aid applications are available through the FAFSA website.
"""

test_prompt = prompt_template.format(
    context=test_context,
    instruction="How much does tuition cost per credit hour?",
    thought="",
    answer=""
)

inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")

print("Testing model...\n")
outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True, temperature=0.7)
response = tokenizer.batch_decode(outputs)[0]

print("=" * 60)
print("RESPONSE:")
print("=" * 60)
print(response)
print("=" * 60)

## 9. Save Fine-tuned Model

In [ ]:
# Save LoRA adapters locally
save_dir = "campus_gpt_lora"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"✅ Model saved to {save_dir}/")
print("\nYou can now:")
print("1. Use it with Ollama: Create a Modelfile")
print("2. Upload to HuggingFace Hub")
print("3. Merge LoRA adapters with base model")

## Optional: Merge LoRA with Base Model

This creates a standalone model (no adapters needed).

In [ ]:
# Merge LoRA adapters into base model
model.save_pretrained_merged(
    "campus_gpt_merged",
    tokenizer,
    save_method="merged_16bit",  # or "merged_4bit" for smaller size
)

print("✅ Merged model saved to campus_gpt_merged/")
print("This can be used with Ollama or deployed directly.")